In [2]:
#from google.colab import drive
import numpy as np
import pandas as pd
import re
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from time import time
import pickle
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from scipy.sparse import hstack

In [3]:
# connect to colab, or you can use local path
#drive.mount('/content/drive/')
#dirpath = '/content/drive/MyDrive/climate change/'
#load data
df = pd.read_csv('convincing_data.csv')

In [6]:
# sentiment score, using -1/3, 1/3 as split line, this way the perfromance is better
def sentimentClassify(sentiment_label):
  if -1 <= sentiment_label < -1+2/3:
    return -1
  elif -1+2/3 <= sentiment_label < -1+4/3:
    return 0
  elif -1+4/3 <= sentiment_label <= 1:
    return 1
df['sentiment_tri'] = df['sentiment'].apply(sentimentClassify)

In [6]:
# using -0.5 0.5 as split line
def map_sentiment_label(sentiment_label):
    if sentiment_label > 0.5:
        return 1
    elif sentiment_label < -0.5:
        return -1
    else:
        return 0
df['sentiment_tri'] = df['sentiment'].apply(map_sentiment_label)

In [7]:
#########################
# process comments so that we can calculate tf-idf
comments = df['body_cleaned']
words_list = comments.apply(lambda x:re.split(r"[\[,'\]\s]+", x)[1:])
wordslist = words_list.apply(lambda x: ' '.join(x))

In [8]:
df['words_list'] = wordslist
Dataset = df[['words_list','score','body_length','sentiment_tri']]
#Dataset.to_csv('DataSet.csv')

In [3]:
# Read the processed data directly if kernel crushes
#Dataset = pd.read_csv('DataSet.csv')
#wordslist = Dataset['words_list']

In [10]:
# calculate tf-idf
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
tfidf_comments = tfidf_vectorizer.fit_transform(wordslist)


In [ ]:
# If use other attributes like body_lenght, score, then use this part

import scipy.sparse as sp
from scipy.sparse import csr_matrix
# transfer score and body_leangth to sparse matrix so that they could concat with tf-idf comments vector
score_sparse = csr_matrix.transpose(sp.csr_matrix(Dataset['score']))
body_length_sparse = csr_matrix.transpose(sp.csr_matrix(Dataset['body_length']))

X_with_attrs = sp.hstack([tfidf_comments, score_sparse, body_length_sparse]) # 将X和attrs_sparse水平合并

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

# Initialize TruncatedSVD model, setting dimensions
svd = TruncatedSVD(n_components=1000)

# 对稀疏矩阵进行降维
X = svd.fit_transform(X_with_attrs)

In [11]:
## If only use comments, then use tf-idf matrix as X data set
X = tfidf_comments

In [12]:
#Y = pd.read_csv('DataSet_Y.csv',index_col=0)
Y = Dataset['sentiment_tri']

In [13]:
#########################
# split data set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [14]:
# LightGBM Training
start_time = time()
from lightgbm import LGBMClassifier
lgbm_clf=LGBMClassifier(boosting_type='gbdt', class_weight=None, 
               learning_rate=0.1, max_depth=6, n_estimators=300, 
               reg_alpha=0.0, reg_lambda=0.0,
               colsample_bytree=1.0, subsample=1.0,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0, num_leaves=100,
               n_jobs=-1, objective='multiclass', importance_type='split',
               random_state=42)
lgbm_clf.fit(X_train,y_train)
print("--- %s seconds ---" % (time() - start_time))
print('Training accuracy {:.4f}'.format(lgbm_clf.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(lgbm_clf.score(X_test,y_test)))

--- 61.95776104927063 seconds ---
Training accuracy 0.7200
Testing accuracy 0.6962


In [15]:
# Naive Bayes model training
start = time()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

print(time()-start)

              precision    recall  f1-score   support

          -1       0.60      0.78      0.68     59343
           0       0.39      0.00      0.00     24013
           1       0.65      0.71      0.68     58492

    accuracy                           0.62    141848
   macro avg       0.54      0.50      0.45    141848
weighted avg       0.58      0.62      0.56    141848

0.35368800163269043
